# db.py Module

Handles database connection, migrations, and utility functions for the Flask app.

In [ ]:
import sqlite3
import os

DB_PATH = os.path.join(os.path.dirname(__file__), 'instance', 'site.db')

def get_db():
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    return conn

def init_db():
    db = get_db()
    try:
        cur = db.cursor()
        # Migration logic for products table
        cur.execute("PRAGMA table_info(products)")
        pcols = [row[1] for row in cur.fetchall()]
        for col in ["image1", "image2", "image3", "image4", "image5"]:
            if col not in pcols:
                cur.execute(f"ALTER TABLE products ADD COLUMN {col} TEXT")
        db.commit()
    finally:
        db.close()
    # ...other migration logic as needed...


# Database Module
All database connection, initialization, migration, and table creation logic from the Flask app.

In [ ]:
import os
import sqlite3
import datetime
from contextlib import closing

APP_DIR = os.path.abspath(os.path.dirname(__file__))
INSTANCE_DIR = os.path.join(APP_DIR, "instance")
os.makedirs(INSTANCE_DIR, exist_ok=True)
DB_PATH = os.path.join(INSTANCE_DIR, "site.db")

def get_db():
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row
    return conn

def get_env(name, default=None):
    val = os.environ.get(name)
    return val if val is not None else default


In [ ]:
def init_db():
    # Ensure image columns exist in products table for existing databases
    db = get_db()
    try:
        cur = db.cursor()
        cur.execute("PRAGMA table_info(products)")
        pcols = [row[1] for row in cur.fetchall()]
        for col in ["image1", "image2", "image3", "image4", "image5"]:
            if col not in pcols:
                cur.execute(f"ALTER TABLE products ADD COLUMN {col} TEXT")
        db.commit()
    finally:
        db.close()
    # Ensure custom_url column exists before upsert
    db = get_db()
    try:
        cur = db.cursor()
        cur.execute("PRAGMA table_info(providers)")
        pcols = [row[1] for row in cur.fetchall()]
        if "custom_url" not in pcols:
            cur.execute("ALTER TABLE providers ADD COLUMN custom_url TEXT")
            db.commit()
            cur.execute("CREATE UNIQUE INDEX IF NOT EXISTS idx_providers_custom_url ON providers(custom_url)")
            db.commit()
    finally:
        db.close()
    # Upsert demo provider with all social media pins for demo
    db = get_db()
    try:
        cur = db.cursor()
        cur.execute("SELECT id FROM providers WHERE custom_url = 'taime-demo'")
        row = cur.fetchone()
        if row:
            cur.execute("""
                UPDATE providers SET
                    linkedin_url = 'https://linkedin.com/in/taime',
                    facebook_url = 'https://facebook.com/taime',
                    instagram_url = 'https://instagram.com/taime',
                    twitter_url = 'https://twitter.com/taime',
                    website_url = 'https://taime.com',
                    youtube_url = 'https://youtube.com/taime'
                WHERE custom_url = 'taime-demo'
            """)
        else:
            cur.execute(
                "INSERT INTO providers (email, password_hash, first_name, business_name, phone, base_zip, address, about, profile_photo, linkedin_url, facebook_url, instagram_url, twitter_url, website_url, youtube_url, custom_url, active, created_at) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                (
                    'demo@provider.com', 'demo', 'Taime', 'Taime Designs', '555-123-4567', '90210', '123 Main St', 'Sample provider with all social pins.', '',
                    'https://linkedin.com/in/taime',
                    'https://facebook.com/taime',
                    'https://instagram.com/taime',
                    'https://twitter.com/taime',
                    'https://taime.com',
                    'https://youtube.com/taime',
                    'taime-demo', 1, datetime.datetime.now().isoformat()
                )
            )
        db.commit()
    finally:
        db.close()


In [ ]:
def create_tables_and_migrate():
    db = get_db()
    try:
        db.executescript(
            """
CREATE TABLE IF NOT EXISTS services (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    description TEXT,
    price TEXT,
    posted_by TEXT,
    provider_id INTEGER NOT NULL DEFAULT 0,
    active INTEGER NOT NULL DEFAULT 1,
    image1 TEXT,
    image2 TEXT,
    image3 TEXT,
    image4 TEXT,
    image5 TEXT,
    is_certified INTEGER DEFAULT 0,
    certification_proof TEXT,
    created_at TEXT NOT NULL
);
CREATE TABLE IF NOT EXISTS blocked_addresses (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    address TEXT NOT NULL,
    zip TEXT,
    message TEXT,
    provider_id INTEGER NOT NULL DEFAULT 0
);
CREATE TABLE IF NOT EXISTS providers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    email TEXT UNIQUE NOT NULL,
    password_hash TEXT NOT NULL,
    first_name TEXT,
    business_name TEXT,
    phone TEXT,
    base_zip TEXT,
    address TEXT,
    about TEXT,
    profile_photo TEXT,
    linkedin_url TEXT,
    facebook_url TEXT,
    instagram_url TEXT,
    twitter_url TEXT,
    website_url TEXT,
    youtube_url TEXT,
    custom_url TEXT UNIQUE,
    active INTEGER NOT NULL DEFAULT 1,
    created_at TEXT NOT NULL
);
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    description TEXT,
    price TEXT,
    provider_id INTEGER NOT NULL DEFAULT 0,
    active INTEGER NOT NULL DEFAULT 1,
    created_at TEXT NOT NULL
);
CREATE TABLE IF NOT EXISTS provider_zips (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    provider_id INTEGER NOT NULL,
    zip_code TEXT NOT NULL,
    radius_miles INTEGER DEFAULT 10,
    created_at TEXT NOT NULL,
    UNIQUE(provider_id, zip_code)
);
CREATE TABLE IF NOT EXISTS events (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    description TEXT,
    date TEXT NOT NULL,
    location TEXT,
    zip TEXT,
    provider_id INTEGER NOT NULL,
    created_at TEXT NOT NULL
);
CREATE TABLE IF NOT EXISTS auth (
    id INTEGER PRIMARY KEY CHECK (id = 1),
    admin_password_hash TEXT,
    provider_password_hash TEXT
);
INSERT OR IGNORE INTO auth (id) VALUES (1);
INSERT OR IGNORE INTO profile (id, business_name)
VALUES (1, 'Your Mom''s Services');
            """
        )
        db.commit()
    finally:
        db.close()


In [ ]:
def alter_and_migrate_columns():
    # Add columns to profile table if missing
    db = get_db()
    try:
        cur = db.cursor()
        cur.execute("PRAGMA table_info(profile)")
        pcols = [row[1] for row in cur.fetchall()]
        if "first_name" not in pcols:
            cur.execute("ALTER TABLE profile ADD COLUMN first_name TEXT")
            db.commit()
        if "profile_photo" not in pcols:
            cur.execute("ALTER TABLE profile ADD COLUMN profile_photo TEXT")
            db.commit()
    finally:
        db.close()
    # Add columns to leads table if missing
    db = get_db()
    try:
        cur = db.cursor()
        cur.execute("PRAGMA table_info(leads)")
        lcols = [row[1] for row in cur.fetchall()]
        if "address" not in lcols:
            cur.execute("ALTER TABLE leads ADD COLUMN address TEXT")
            db.commit()
        if "status" not in lcols:
            cur.execute("ALTER TABLE leads ADD COLUMN status TEXT DEFAULT 'new'")
            db.commit()
        if "follow_up_date" not in lcols:
            cur.execute("ALTER TABLE leads ADD COLUMN follow_up_date TEXT")
            db.commit()
        if "recurring" not in lcols:
            cur.execute("ALTER TABLE leads ADD COLUMN recurring INTEGER DEFAULT 0")
            db.commit()
        if "provider_id" not in lcols:
            cur.execute("ALTER TABLE leads ADD COLUMN provider_id INTEGER DEFAULT 0")
            db.commit()
    finally:
        db.close()


In [ ]:
def rebuild_fts_if_needed(app_has_fts):
    if app_has_fts:
        db = get_db()
        try:
            cur = db.cursor()
            cur.execute("INSERT INTO services_fts(services_fts) VALUES ('rebuild')")
            db.commit()
        except sqlite3.OperationalError:
            pass
        finally:
            db.close()


In [ ]:
def add_social_media_columns():
    with closing(get_db()) as db:
        try:
            db.execute("ALTER TABLE providers ADD COLUMN linkedin_url TEXT")
            db.execute("ALTER TABLE providers ADD COLUMN facebook_url TEXT")
            db.execute("ALTER TABLE providers ADD COLUMN instagram_url TEXT")
            db.execute("ALTER TABLE providers ADD COLUMN twitter_url TEXT")
            db.execute("ALTER TABLE providers ADD COLUMN website_url TEXT")
            db.execute("ALTER TABLE providers ADD COLUMN youtube_url TEXT")
            db.commit()
            print("Added social media columns to providers table")
        except sqlite3.OperationalError:
            # Columns already exist
            pass


In [ ]:
def resolve_password_hashes():
    with closing(get_db()) as db:
        cur = db.cursor()
        cur.execute("SELECT admin_password_hash, provider_password_hash FROM auth WHERE id = 1")
        row = cur.fetchone()

        # Resolve admin hash
        admin_hash_db = row["admin_password_hash"] if row else None
        if not admin_hash_db:
            admin_password_hash = get_env("ADMIN_PASSWORD_HASH")
            admin_password_plain = get_env("ADMIN_PASSWORD")
            if admin_password_plain and not admin_password_hash:
                admin_password_hash = generate_password_hash(admin_password_plain)
            if not admin_password_hash:
                print("WARNING: ADMIN_PASSWORD/ADMIN_PASSWORD_HASH not set. Defaulting to 'changeme'.")
                admin_password_hash = generate_password_hash("changeme")
            cur.execute("UPDATE auth SET admin_password_hash = ? WHERE id = 1", (admin_password_hash,))
            db.commit()
            admin_hash_final = admin_password_hash
        else:
            admin_hash_final = admin_hash_db

        # Resolve provider hash (defaults to admin if missing)
        provider_hash_db = row["provider_password_hash"] if row else None
        if not provider_hash_db:
            provider_password_hash = get_env("PROVIDER_PASSWORD_HASH")
            provider_password_plain = get_env("PROVIDER_PASSWORD")
            if provider_password_plain and not provider_password_hash:
                provider_password_hash = generate_password_hash(provider_password_plain)
            if not provider_password_hash:
                provider_password_hash = admin_hash_final
                print("INFO: PROVIDER_PASSWORD not set; provider login uses the admin password.")
            cur.execute("UPDATE auth SET provider_password_hash = ? WHERE id = 1", (provider_password_hash,))
            db.commit()
